In [1]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/P1015.3. Analyses/P1015.3.2. Data Processing/Output'
cur_date = '040825'

library(missForest)
library(readxl)
library(openxlsx)
library(tidyverse)
library(imputeLCMD)
library(factoextra)

# reading in files
ws_df = data.frame(Data = 'WS', read_excel("Input/Woodsmoke_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)
wf_df = data.frame(Data = 'WF', read_excel("Input/Wildfire_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)
control_df = data.frame(read_excel("Input/Control_Data_032825.xlsx", sheet = 2))  %>%
    select(-Sample_Number)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
an

In [2]:
head(ws_df)
head(wf_df)
head(control_df)

,Data,HAWC_ID,Study,Replicate,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,1263480,Burnet et al. 1990,NA,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


,Data,HAWC_ID,Study,Replicate,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289821,Liang et al. 2021,1,PAH,Volume,NA,1-(10-methylanthracen-9-yl)ethanone,5
2,WF,1289821,Liang et al. 2021,1,PAH,Volume,DTXSID50176885,1-Acenaphthenone,1
3,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
4,WF,1289739,Campos et al. 2019,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
5,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
6,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA


,Data,HAWC_ID,Study,Replicate,Class,Metric,DTXSID,Name,Value
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID3021774,Acenaphthene,71.17
2,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID3023845,Acenaphthylene,19.63
3,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID5023902,Benzo(a)anthracene,333.74
4,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID2020139,Benzo(a)pyrene,1526.38
5,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID80175664,Benzo[b]chrysene,988.96
6,TRAP,1298357,Lee at al. 1995,1,PAH,Weight,DTXSID0023907,Benzo(b)fluoranthene,1334.97


In [3]:
dim(ws_df)
dim(wf_df)
dim(control_df)

[1] 6607    9

[1] 4340    9

[1] 1244    9

# Filter #1

Removing duplicate records for instances that weren't measured at all (MAR). 

In [4]:
`%notin%` <- Negate(`%in%`)

ws_df = ws_df %>%
    filter(Replicate %notin% NA)
wf_df = wf_df %>%
    filter(Replicate %notin% NA)
control_df = control_df %>%
    filter(Replicate %notin% NA)

In [5]:
dim(ws_df)
dim(wf_df)
dim(control_df)

[1] 4203    9

[1] 2378    9

[1] 1215    9

In [6]:
# combining dfs
combined_df = rbind(ws_df, wf_df, control_df)
head(combined_df)

,Data,HAWC_ID,Study,Replicate,Class,Metric,DTXSID,Name,Value
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,267091,Forchhammer et al. 2012,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


In [7]:
length(unique(combined_df$Name))
length(unique(ws_df$Name))
length(unique(wf_df$Name))
length(unique(control_df$Name))

[1] 228

[1] 137

[1] 138

[1] 127

228 unique metals/chemicals total.

# Filter #2: Chemical/Metal Background Filters

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. 

A variable (ie. `Name` which represents a chemical or metal) will be retained if >= 50% of its experimental data points were measurable values. In other words, chemicals were excluded if they were NAs (not evaluated) or had non-detect values across < 50% of the samples.

In [8]:
sample_type_presence_df = combined_df %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least 50% experimental values for each chemical/metal
    group_by(Metric, Class, Data, Name) %>%
    # calculating the percentage of experimental data points
    reframe(Variable_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Variable_Presence_Percentage >= 50) 

head(sample_type_presence_df)

Metric,Class,Data,Name,Variable_Presence_Percentage
<chr>,<chr>,<chr>,<chr>,<dbl>
Volume,Metal,Diesel,Al,100
Volume,Metal,Diesel,As,100
Volume,Metal,Diesel,Ca,100
Volume,Metal,Diesel,Cd,100
Volume,Metal,Diesel,Cr,100
Volume,Metal,Diesel,Cu,100


In [9]:
# filtering the original df
filter1_df = inner_join(sample_type_presence_df[,1:4], combined_df) %>%
    # metals don't have a DTXSID, so making that col their name
    mutate(DTXSID = ifelse(DTXSID != 'NA', DTXSID, Name)) 

head(filter1_df)

Joining with `by = join_by(Metric, Class, Data, Name)`


Metric,Class,Data,Name,HAWC_ID,Study,Replicate,DTXSID,Value
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
Volume,Metal,Diesel,Al,1298361,Loyola et al. 2009,1,Al,4811
Volume,Metal,Diesel,Al,1298361,Loyola et al. 2009,2,Al,4160
Volume,Metal,Diesel,As,1298371,Sanchez Jimenez et al. 2012,1,As,7.0000000000000007E-2
Volume,Metal,Diesel,As,1298371,Sanchez Jimenez et al. 2012,2,As,0.04
Volume,Metal,Diesel,As,1298371,Sanchez Jimenez et al. 2012,4,As,0.44
Volume,Metal,Diesel,As,1298371,Sanchez Jimenez et al. 2012,5,As,0.52400000000000002


In [10]:
dim(combined_df)
dim(filter1_df)

[1] 7796    9

[1] 5335    9

Started with 7796 records, 2461 were removed, leaving 5335. 

# Filter 3: Study Background Filter

A study will be retained if >= 50% of its experimental data points were measurable values. In other words, chemicals were excluded if they were NAs (not evaluated) or had non-detect values across < 50% of the samples.

In [11]:
study_type_presence_df = filter1_df %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least 50% experimental values for each chemical/metal
    group_by(Metric, Class, HAWC_ID) %>%
    # calculating the percentage of experimental data points
    reframe(Study_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Study_Presence_Percentage >= 20) 

head(study_type_presence_df)

Metric,Class,HAWC_ID,Study_Presence_Percentage
<chr>,<chr>,<chr>,<dbl>
Volume,Metal,1070627,100
Volume,Metal,1263480,100
Volume,Metal,1263484,100
Volume,Metal,1289878,100
Volume,Metal,1289884,100
Volume,Metal,1289885,100


In [12]:
filter2_df = inner_join(study_type_presence_df[,1:3], filter1_df)

head(filter2_df)

Joining with `by = join_by(Metric, Class, HAWC_ID)`


Metric,Class,HAWC_ID,Data,Name,Study,Replicate,DTXSID,Value
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
Volume,Metal,1070627,WF,As,Siregar et al. 2022,1,As,2.2999999999999998
Volume,Metal,1070627,WF,As,Siregar et al. 2022,2,As,0.8
Volume,Metal,1070627,WF,Ca,Siregar et al. 2022,1,Ca,1370.1
Volume,Metal,1070627,WF,Ca,Siregar et al. 2022,2,Ca,595.20000000000005
Volume,Metal,1070627,WF,Cd,Siregar et al. 2022,1,Cd,8.8000000000000007
Volume,Metal,1070627,WF,Cd,Siregar et al. 2022,2,Cd,2.1


In [13]:
dim(filter1_df)
dim(filter2_df)

[1] 5335    9

[1] 5305    9

299 more records were removed.

In [14]:
# recombining data
filter2_split_df = filter2_df %>%
    group_by(Metric) %>%
    group_split()

vol_df = filter2_split_df[[1]]
weight_df = filter2_split_df[[2]]

head(vol_df)

Metric,Class,HAWC_ID,Data,Name,Study,Replicate,DTXSID,Value
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
Volume,Metal,1070627,WF,As,Siregar et al. 2022,1,As,2.2999999999999998
Volume,Metal,1070627,WF,As,Siregar et al. 2022,2,As,0.8
Volume,Metal,1070627,WF,Ca,Siregar et al. 2022,1,Ca,1370.1
Volume,Metal,1070627,WF,Ca,Siregar et al. 2022,2,Ca,595.20000000000005
Volume,Metal,1070627,WF,Cd,Siregar et al. 2022,1,Cd,8.8000000000000007
Volume,Metal,1070627,WF,Cd,Siregar et al. 2022,2,Cd,2.1


# Square Root Imputation

Taking the square root of the min value to impute MNAR data.

In [15]:
# first determining number of ND and NA values in each df
vol_df %>%
    filter(Value %in% c('NA', 'ND')) %>%
    group_by(Value) %>%
    summarize(n())

weight_df %>%
    filter(Value %in% c('NA', 'ND')) %>%
    group_by(Value) %>%
    summarize(n())

Value,n()
<chr>,<int>
NA,18
ND,27


Value,n()
<chr>,<int>
NA,238
ND,140


In [16]:
dim(vol_df)
dim(weight_df)

[1] 1918    9

[1] 3387    9

In [17]:
# creating a df for MAR data
mar_weight_df = weight_df %>%
    filter(Value == 'NA')
mar_vol_df = vol_df %>%
    filter(Value == 'NA')

preimputed_weight_df = anti_join(weight_df, mar_weight_df) 
preimputed_vol_df = anti_join(vol_df, mar_vol_df) 

preimputed_weight_df$Value = as.numeric(preimputed_weight_df$Value)
preimputed_vol_df$Value = as.numeric(preimputed_vol_df$Value)
mar_weight_df$Value = as.numeric(mar_weight_df$Value)
mar_vol_df$Value = as.numeric(mar_vol_df$Value)

# reordering cols
preimputed_weight_df = preimputed_weight_df[,c(3,1,4,6,7,2,8,5,9)]
preimputed_vol_df = preimputed_vol_df[,c(3,1,4,6,7,2,8,5,9)]

head(preimputed_weight_df)

Joining with `by = join_by(Metric, Class, HAWC_ID, Data, Name, Study,
Replicate, DTXSID, Value)`
Joining with `by = join_by(Metric, Class, HAWC_ID, Data, Name, Study,
Replicate, DTXSID, Value)`
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”


HAWC_ID,Metric,Data,Study,Replicate,Class,DTXSID,Name,Value
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Al,Al,1078
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Al,Al,198
1040882,Weight,WS,Nordin et al. 2015,1,Metal,B,B,308
1040882,Weight,WS,Nordin et al. 2015,2,Metal,B,B,268
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Ca,Ca,7120
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Ca,Ca,NA


In [18]:
# creating a df for MNAR data
mnar_weight_df = preimputed_weight_df %>%
    filter(is.na(Value))
mnar_vol_df = preimputed_vol_df %>%
    filter(is.na(Value))

head(mnar_weight_df)

HAWC_ID,Metric,Data,Study,Replicate,Class,DTXSID,Name,Value
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Ca,Ca,NA
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Fe,Fe,NA
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Mn,Mn,NA
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Mn,Mn,NA
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Ni,Ni,NA
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Ni,Ni,NA


In [19]:
square_root_imputation = function(mnar_df, preimputed_df){
    # ADD WORDS

    id_df = mnar_df[is.na(mnar_df$Value),]
    study_ids = unique(id_df$HAWC_ID)
    
    imputed_df = data.frame()
    for (i in 1:length(id_df$HAWC_ID)){
    
        filtered_df = preimputed_df %>%
            filter(HAWC_ID == id_df$HAWC_ID[i], DTXSID == id_df$DTXSID[i])
    
        na_df = filtered_df %>%
            filter(is.na(Value)) %>%
            mutate(Value = sqrt(min(filtered_df$Value, na.rm = TRUE)))
    
        # combining rows of data
        combined_df = rbind(filtered_df %>%
            filter(!is.na(Value)), na_df)
    
        imputed_df = rbind(imputed_df, combined_df) %>%
                        unique()
    }
    return(imputed_df)    
    
}

# calling fn
imputed_vol_df = square_root_imputation(mnar_vol_df, preimputed_vol_df) %>%
    mutate(Value = ifelse(Value == Inf, NA, Value))
imputed_weight_df = square_root_imputation(mnar_weight_df, preimputed_weight_df) %>%
    mutate(Value = ifelse(Value == Inf, NA, Value))
    
head(imputed_weight_df)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; retur

HAWC_ID,Metric,Data,Study,Replicate,Class,DTXSID,Name,Value
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Ca,Ca,7120.00000
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Ca,Ca,84.38009
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Fe,Fe,1172.00000
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Fe,Fe,34.23449
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Mn,Mn,NA
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Mn,Mn,NA


In [20]:
# there are still a few variables that didn't have any values to do the square root imputation
# their imputed values will be based off of distributions from other studies
other_studies_vol_df = semi_join(preimputed_vol_df %>%
                filter(!is.na(Value)), imputed_vol_df %>%
                               filter(is.na(Value)) %>%
    select(2,3,6,7)) %>%
    group_by(DTXSID) %>%
    summarize(Value = sqrt(min(Value)))

other_studies_weight_df = semi_join(preimputed_weight_df %>%
                filter(!is.na(Value)), imputed_weight_df %>%
                               filter(is.na(Value)) %>%
    select(2,3,6,7)) %>%
    group_by(DTXSID) %>%
    summarize(Value = sqrt(min(Value)))

head(other_studies_vol_df)
head(other_studies_weight_df)

Joining with `by = join_by(Metric, Data, Class, DTXSID)`
Joining with `by = join_by(Metric, Data, Class, DTXSID)`


DTXSID,Value
<chr>,<dbl>
DTXSID2020139,0.6633250
DTXSID3021774,0.2828427
DTXSID5023908,1.1916375


DTXSID,Value
<chr>,<dbl>
Al,1.31148770
Cr,0.31622777
DTXSID0023909,0.05234501
DTXSID3023845,0.11584472
DTXSID4047753,0.02701851
DTXSID8020913,0.15023315


In [21]:
# removing NAs from the imputed df 
removed_imputed_vol_df = imputed_vol_df %>%
    filter(!is.na(Value))
removed_imputed_weight_df = imputed_weight_df %>%
    filter(!is.na(Value))

# adding in the newly imputed values back in 
all_imputed_vol_df = cbind(imputed_vol_df %>%
    filter(is.na(Value)) %>%
    select(-Value), Value = c(rep(other_studies_vol_df$Value[3],2), rep(other_studies_vol_df$Value[1],2),
                     rep(other_studies_vol_df$Value[2],2)))
all_imputed_vol_df = rbind(removed_imputed_vol_df, all_imputed_vol_df)

all_imputed_weight_df = cbind(imputed_weight_df %>%
    filter(is.na(Value)) %>%
    select(-Value), Value = c(rep(other_studies_weight_df$Value[10],2), rep(other_studies_weight_df$Value[11],2),
                     rep(other_studies_weight_df$Value[10],2), rep(other_studies_weight_df$Value[12],2), rep(NA,5),
                     other_studies_weight_df$Value[c(1:2,9,11,12,8)], rep(other_studies_weight_df$Value[4],3),
                     rep(other_studies_weight_df$Value[6],3), other_studies_weight_df$Value[8], 
                     rep(other_studies_weight_df$Value[7],3), rep(other_studies_weight_df$Value[5],3)))

all_imputed_weight_df = rbind(removed_imputed_vol_df, all_imputed_weight_df)
# left_join(imputed_df %>%
#         filter(is.na(Value)) %>%
#         select(-Value), other_studies_df)

head(all_imputed_weight_df)

HAWC_ID,Metric,Data,Study,Replicate,Class,DTXSID,Name,Value
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1298351,Volume,TRAP,Gunawardena et al. 2012,3,Metal,Cr,Cr,20
1298351,Volume,TRAP,Gunawardena et al. 2012,4,Metal,Cr,Cr,30
1298351,Volume,TRAP,Gunawardena et al. 2012,6,Metal,Cr,Cr,30
1298351,Volume,TRAP,Gunawardena et al. 2012,7,Metal,Cr,Cr,90
1298351,Volume,TRAP,Gunawardena et al. 2012,10,Metal,Cr,Cr,80
1298351,Volume,TRAP,Gunawardena et al. 2012,11,Metal,Cr,Cr,20


In [22]:
# creating 1 df
mnar_imputed_vol_df = rbind(#imputed data
                               all_imputed_vol_df,
                               # adding in MAR data
                               mar_vol_df,
                               # adding back in non-imputed data
                               anti_join(preimputed_vol_df %>%
                                         filter(HAWC_ID %notin% all_imputed_vol_df$HAWC_ID), mnar_vol_df))
mnar_imputed_weight_df = rbind(# imputed data
                               all_imputed_weight_df,
                               # adding in MAR data
                               mar_weight_df,
                               # adding back in non-imputed data
                               anti_join(preimputed_weight_df %>%
                                         filter(HAWC_ID %notin% all_imputed_weight_df$HAWC_ID), mnar_weight_df))

head(mnar_imputed_weight_df)
dim(mnar_imputed_vol_df)
dim(mnar_imputed_weight_df)

Joining with `by = join_by(HAWC_ID, Metric, Data, Study, Replicate, Class,
DTXSID, Name, Value)`
Joining with `by = join_by(HAWC_ID, Metric, Data, Study, Replicate, Class,
DTXSID, Name, Value)`


HAWC_ID,Metric,Data,Study,Replicate,Class,DTXSID,Name,Value
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1298351,Volume,TRAP,Gunawardena et al. 2012,3,Metal,Cr,Cr,20
1298351,Volume,TRAP,Gunawardena et al. 2012,4,Metal,Cr,Cr,30
1298351,Volume,TRAP,Gunawardena et al. 2012,6,Metal,Cr,Cr,30
1298351,Volume,TRAP,Gunawardena et al. 2012,7,Metal,Cr,Cr,90
1298351,Volume,TRAP,Gunawardena et al. 2012,10,Metal,Cr,Cr,80
1298351,Volume,TRAP,Gunawardena et al. 2012,11,Metal,Cr,Cr,20


[1] 1578    9

[1] 2897    9

In [23]:
# dim(vol_df)
# dim(mar_vol_df)
# dim(mnar_vol_df)
# 1378 - 8 - 25
# dim(anti_join(preimputed_vol_df, mnar_vol_df)) 

# dim(all_imputed_vol_df) # yes!
# dim(imputed_vol_df)
# dim(preimputed_vol_df %>%
#                   filter(HAWC_ID %notin% all_imputed_vol_df$HAWC_ID))
# 1095 + 63 

# Filter #4: Double Checking MAR Records

Making sure that when the data is split based on `Class` and transformed into a wide format, we have at least 50% of the records with numeric values for each metal or chemical.

In [24]:
split_vol_df = mnar_imputed_vol_df %>%
    # averaging to get rid of replicates
    group_by(HAWC_ID, DTXSID) %>%
    mutate(Avg_Value = mean(Value, na.rm = TRUE)) %>%
    ungroup() %>%
    select(-c("Replicate", "Value", "Name")) %>%
    unique() %>%
    group_by(Class) %>%
    group_split

metal_vol_df = split_vol_df[[1]]
pah_vol_df = split_vol_df[[2]]
voc_vol_df = split_vol_df[[3]]

head(metal_vol_df)

HAWC_ID,Metric,Data,Study,Class,DTXSID,Avg_Value
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1298351,Volume,TRAP,Gunawardena et al. 2012,Metal,Cr,28.78885
1298351,Volume,TRAP,Gunawardena et al. 2012,Metal,Cu,13.30890
USEPA,Volume,TRAP,USEPA,Metal,Cu,NaN
1070627,Volume,WF,Siregar et al. 2022,Metal,As,1.55000
1070627,Volume,WF,Siregar et al. 2022,Metal,Ca,982.65000
1070627,Volume,WF,Siregar et al. 2022,Metal,Cd,5.45000


In [25]:
split_weight_df = mnar_imputed_weight_df %>%
    # averaging to get rid of replicates
    group_by(HAWC_ID, DTXSID) %>%
    mutate(Avg_Value = mean(Value, na.rm = TRUE)) %>%
    ungroup() %>%
    select(-c("Replicate", "Value", "Name")) %>%
    unique() %>%
    group_by(Class) %>%
    group_split

metal_weight_df = split_weight_df[[1]]
pah_weight_df = split_weight_df[[2]]

head(metal_weight_df)

HAWC_ID,Metric,Data,Study,Class,DTXSID,Avg_Value
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1298351,Volume,TRAP,Gunawardena et al. 2012,Metal,Cr,28.7888544
1298351,Volume,TRAP,Gunawardena et al. 2012,Metal,Cu,13.3088968
1040882,Weight,WS,Nordin et al. 2015,Metal,Mn,0.4582576
1040882,Weight,WS,Nordin et al. 2015,Metal,Ni,0.1732051
267105,Weight,WS,Danielsen et al. 2011,Metal,Mn,0.4582576
271638,Weight,WS,Jalava et al. 2012,Metal,Pb,0.9380832


In [26]:
# final_df = rbind(mnar_imputed_vol_df %>%
#     # averaging to get rid of replicates
#     group_by(HAWC_ID, DTXSID) %>%
#     mutate(Avg_Value = mean(Value, na.rm = TRUE)) %>%
#     ungroup() %>%
#     select(-c("Replicate", "Value")) %>%
#     unique(),
#     mnar_imputed_weight_df %>%
#     # averaging to get rid of replicates
#     group_by(HAWC_ID, DTXSID) %>%
#     mutate(Avg_Value = mean(Value, na.rm = TRUE)) %>%
#     ungroup() %>%
#     select(-c("Replicate", "Value")) %>%
#     unique()) %>% unique()

In [27]:
# # exporting
# write.xlsx(final_df, paste0(Output,"/", "Processed_Data_", cur_date, ".xlsx"), 
#            rowNames = FALSE)

Converting values to 0s and 1s to count number of experimental and NDs previously imputed. Chemicals/metals that don't have at least 50% of values present will be removed.

In [28]:
wider_metal_vol_df = metal_vol_df %>%
    # creating sample ids
    unite("SampleID", c(colnames(metal_vol_df)[c(1,3,5)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value") 
wider_pah_vol_df = pah_vol_df %>%
    # creating sample ids
    unite("SampleID", c(colnames(pah_vol_df)[c(1,3,5)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value")
wider_voc_vol_df = voc_vol_df %>%
    # creating sample ids
    unite("SampleID", c(colnames(pah_vol_df)[c(1,3,5)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value")

wider_metal_weight_df = metal_weight_df %>%
    unite("SampleID", c(colnames(metal_weight_df)[c(1,3,5)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value") 
wider_pah_weight_df = pah_weight_df %>%
    unite("SampleID", c(colnames(pah_weight_df)[c(1,3,5)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value")  

head(wider_metal_weight_df)

SampleID,HAWC_ID,Metric,Data,Study,Class,Cr,Cu,Mn,Ni,⋯,Ta,Tb,Tm,Yb,Mo,Sn,As,Sb,V,Zr
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1298351_TRAP_Metal,1298351,Volume,TRAP,Gunawardena et al. 2012,Metal,28.7888544,13.3089,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1040882_WS_Metal,1040882,Weight,WS,Nordin et al. 2015,Metal,NA,NA,0.4582576,0.1732051,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
267105_WS_Metal,267105,Weight,WS,Danielsen et al. 2011,Metal,NA,NA,0.4582576,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
271638_WS_Metal,271638,Weight,WS,Jalava et al. 2012,Metal,NaN,NA,NA,NaN,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
822010_WS_Metal,822010,Weight,WS,Farina et al. 2019,Metal,0.3162278,NA,NA,0.1732051,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1063054_WS_Metal,1063054,Weight,WS,Corsini et al. 2013,Metal,40.0000000,375.0000,945.0000000,30.0000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [29]:
data_percentage = function(wider_dataset){
    # ADD WORDS!!
    
    # converting values to 0s and 1s
    summed_df = wider_dataset %>%
        mutate(across(7:ncol(wider_dataset), ~ifelse(is.na(.x), 0, 1)))

    # identifying metals/chemicals w/ =< 20% of experimental data
    var_percentage_df = data.frame(Percent = colSums(summed_df[,7:ncol(summed_df)])/nrow(summed_df) * 100) %>%
        filter(Percent <= 20)
    # identifying studies w/ =< 20% of experimental data
    study_percentage_df = data.frame(HAWC_ID = wider_dataset$HAWC_ID,
                                     Percent = rowSums(summed_df[,7:ncol(summed_df)])/ncol(summed_df) * 100) %>%
        filter(Percent <= 20)

    return(list(var_percentage_df, study_percentage_df))
}

# calling fn
percentage_metal_vol_df = data_percentage(wider_metal_vol_df)
percentage_pah_vol_df = data_percentage(wider_pah_vol_df) 
percentage_voc_vol_df = data_percentage(wider_voc_vol_df) 
percentage_metal_weight_df = data_percentage(wider_metal_weight_df) 
percentage_pah_weight_df = data_percentage(wider_pah_weight_df)

head(percentage_metal_weight_df[[1]])

,Percent
,<dbl>
Cd,16.326531
Na,10.204082
B,8.163265
La,8.163265
Sc,8.163265
Sr,18.367347


In [30]:
# removing those metals/chemicals & studies from the OG df
preimputed_metal_vol_df = wider_metal_vol_df[,colnames(wider_metal_vol_df) %notin% rownames(percentage_metal_vol_df[[1]])]
preimputed_pah_vol_df = wider_pah_vol_df[,colnames(wider_pah_vol_df) %notin% rownames(percentage_pah_vol_df[[1]])]
preimputed_voc_vol_df = wider_voc_vol_df[,colnames(wider_voc_vol_df) %notin% rownames(percentage_voc_vol_df[[1]])]
preimputed_metal_weight_df = wider_metal_weight_df[,colnames(wider_metal_weight_df) %notin% rownames(percentage_metal_weight_df[[1]])]
preimputed_pah_weight_df = wider_pah_weight_df[,colnames(wider_pah_weight_df) %notin% rownames(percentage_pah_weight_df[[1]])]

preimputed_metal_vol_df = preimputed_metal_vol_df %>% filter(HAWC_ID %notin% percentage_metal_vol_df[[2]]$HAWC_ID) %>%
    filter(HAWC_ID %notin% c('1298353', '1298372'))
preimputed_pah_vol_df = preimputed_pah_vol_df %>% filter(HAWC_ID %notin% percentage_pah_vol_df[[2]]$HAWC_ID)
#preimputed_voc_vol_df = preimputed_voc_vol_df %>% filter(HAWC_ID %notin% percentage_voc_vol_df[[2]]$HAWC_ID) 
preimputed_metal_weight_df = preimputed_metal_weight_df %>% filter(HAWC_ID %notin% percentage_metal_weight_df[[2]]$HAWC_ID) %>%
    # removing certain variables and studies that were missed
    select(-Pb) %>%
    filter(HAWC_ID != '1289920')
preimputed_pah_weight_df = preimputed_pah_weight_df %>% filter(HAWC_ID %notin% percentage_pah_weight_df[[2]]$HAWC_ID)

head(preimputed_metal_weight_df)

SampleID,HAWC_ID,Metric,Data,Study,Class,Cr,Cu,Mn,Ni,Al,Fe,Ca,Mg,Co,Ba,Zn,K
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1298343_Diesel_Metal,1298343,Weight,Diesel,Bhavaraju et al. 2014,Metal,2.0000,33.000,4.0,3.00000,107,90,367.00,49,2.000000,7.0000,92.50,6.5
1104106_WF_Metal,1104106,Weight,WF,Alexakis 2020,Metal,61.5000,31.250,532.5,125.00000,14500,19200,158250.00,33050,10.500000,117.5000,206.75,4450.0
1257076_WF_Metal,1257076,Weight,WF,Villaruel et al. 2024,Metal,9.0000,693.000,214.0,33.00000,8792,7526,85559.00,4263,8.000000,NA,1276.00,2087.0
1285775_WF_Metal,1285775,Weight,WF,Plumlee et al. 2007,Metal,222.5500,1245.000,585.0,39.10000,58400,37400,160500.00,9320,19.750000,699.5000,2510.00,13850.0
1285933_WF_Metal,1285933,Weight,WF,Alshehri et al. 2023,Metal,243.3333,6574.675,7080.0,167.31667,NA,NA,34396.67,NA,17.346667,954.7967,20554.15,NA
1289708_WF_Metal,1289708,Weight,WF,Alves et al. 2011,Metal,52.8175,NA,NA,47.42833,NA,NA,NA,NA,6.318333,NA,NA,NA


In [31]:
# How many metals/chemicals are now left in the dataset?
ncol(preimputed_metal_vol_df) - 6
ncol(preimputed_pah_vol_df) - 6
ncol(preimputed_voc_vol_df) - 6
ncol(preimputed_metal_weight_df) - 6
ncol(preimputed_pah_weight_df) - 6

[1] 19

[1] 20

[1] 13

[1] 12

[1] 18

In [32]:
# How many studies are left?
nrow(preimputed_metal_vol_df) 
nrow(preimputed_pah_vol_df) 
nrow(preimputed_voc_vol_df) 
nrow(preimputed_metal_weight_df) 
nrow(preimputed_pah_weight_df) 

[1] 15

[1] 11

[1] 17

[1] 11

[1] 12

ADD WORDS. 

# RF Imputation

In [33]:
random_forest_imputation = function(dataset){
    # ADD WORDS
    
    preimputed_df = dataset[,7:ncol(dataset)]
    
    imputed_RF_object = missForest(as.matrix(preimputed_df))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(dataset[,1:6], imputed_RF_df)) %>%
        pivot_longer(cols = 7:ncol(dataset), names_to = "DTXSID", values_to = "Value")
    
    return(imputed_final_df)
}

# calling fn
rf_metal_vol_imputed_df = random_forest_imputation(preimputed_metal_vol_df) 
rf_pah_vol_imputed_df = random_forest_imputation(preimputed_pah_vol_df)
rf_voc_vol_imputed_df = random_forest_imputation(preimputed_voc_vol_df)
rf_metal_weight_imputed_df = random_forest_imputation(preimputed_metal_weight_df) 
rf_pah_weight_imputed_df = random_forest_imputation(preimputed_pah_weight_df)

rf_imputed_df = rbind(rf_metal_vol_imputed_df, rf_pah_vol_imputed_df, 
                      rf_voc_vol_imputed_df, rf_metal_weight_imputed_df,
                      rf_pah_weight_imputed_df
                     ) 

head(rf_imputed_df)
dim(rf_imputed_df)

Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response h

SampleID,HAWC_ID,Metric,Data,Study,Class,DTXSID,Value
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
USEPA_TRAP_Metal,USEPA,Volume,TRAP,USEPA,Metal,Cr,3.91325
USEPA_TRAP_Metal,USEPA,Volume,TRAP,USEPA,Metal,Cu,33.94773
USEPA_TRAP_Metal,USEPA,Volume,TRAP,USEPA,Metal,As,0.18050
USEPA_TRAP_Metal,USEPA,Volume,TRAP,USEPA,Metal,Ca,1104.97117
USEPA_TRAP_Metal,USEPA,Volume,TRAP,USEPA,Metal,Cd,0.09450
USEPA_TRAP_Metal,USEPA,Volume,TRAP,USEPA,Metal,Co,1.32040


[1] 1074    8

In [41]:
# adding back in name col
final_df = left_join(rf_imputed_df, 
                      unique(combined_df[,5:8]) %>%
    # metals don't have a DTXSID, so making that col their name
    mutate(DTXSID = ifelse(grepl("NA", DTXSID), Name, DTXSID)) ) %>%
    unique()

final_df = final_df[,c(2:7,9,8)]

head(final_df)
dim(final_df)

Joining with `by = join_by(Metric, Class, DTXSID)`
Warning message in left_join(rf_imputed_df, unique(combined_df[, 5:8]) %>% mutate(DTXSID = ifelse(grepl("NA", :
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 286 of `x` matches multiple rows in `y`.
ℹ Row 131 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


HAWC_ID,Metric,Data,Study,Class,DTXSID,Name,Value
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
USEPA,Volume,TRAP,USEPA,Metal,Cr,Cr,3.91325
USEPA,Volume,TRAP,USEPA,Metal,Cu,Cu,33.94773
USEPA,Volume,TRAP,USEPA,Metal,As,As,0.18050
USEPA,Volume,TRAP,USEPA,Metal,Ca,Ca,1104.97117
USEPA,Volume,TRAP,USEPA,Metal,Cd,Cd,0.09450
USEPA,Volume,TRAP,USEPA,Metal,Co,Co,1.32040


[1] 1258    8

In [40]:
unique(combined_df[,5:8]) %>%
    # metals don't have a DTXSID, so making that col their name
    mutate(DTXSID = ifelse(grepl("NA", DTXSID), Name, DTXSID))

,Class,Metric,DTXSID,Name
,<chr>,<chr>,<chr>,<chr>
1,PAH,Weight,DTXSID3074787,1-Methylanthracene
4,PAH,Volume,DTXSID3074787,1-Methylanthracene
15,PAH,Volume,DTXSID1074759,1-Methylchrysene
16,PAH,Weight,DTXSID1074759,1-Methylchrysene
52,PAH,Volume,DTXSID00180538,1-Methylfluoranthene
54,PAH,Weight,DTXSID00180538,1-Methylfluoranthene
66,PAH,Weight,DTXSID4061919,1-Methylfluorene
69,PAH,Volume,DTXSID4061919,1-Methylfluorene
80,PAH,Weight,DTXSID9020877,1-Methylnaphthalene


In [35]:
# exporting
write.xlsx(final_df, paste0(Output,"/", "Processed_Data_", cur_date, ".xlsx"), 
           rowNames = FALSE)